# SBTi-Finance Tool - Target Reporting
This notebook is an example of how to use the tool to calculate your portfolio's temperature scores and to save results to be submitted to SBTi's Target Validation Team (TVT) for validation and approval of your own Science Based Target. This notebook generates two deliverables to submit to the TVT.

1. Text formatted report in the 'Checks' section of the notebook
2. Two anonymized excel workbooks for reproducing the portfolio temperature scores

Please see the [methodology](https://sciencebasedtargets.org/wp-content/uploads/2020/09/Temperature-Rating-Methodology-V1.pdf), [guidance](https://sciencebasedtargets.org/wp-content/uploads/2020/10/Financial-Sector-Science-Based-Targets-Guidance-Pilot-Version.pdf) and the [technical documentation](https://sciencebasedtargets.github.io/SBTi-finance-tool/) for more details on how to calculate portfolio temperature scores and how to submit targets to the SBTi for validation and approval. 

See 1_analysis_example (on [Colab](https://colab.research.google.com/github/ScienceBasedTargets/SBTi-finance-tool/blob/main/examples/1_analysis_example.ipynb) or [Github](https://github.com/ScienceBasedTargets/SBTi-finance-tool/blob/main/examples/1_analysis_example.ipynb)) for more in depth example of how to work with Jupyter Notebooks in general and SBTi notebooks in particular. 

## Setting up
This section imports SBTi tools, import data providers, and load the portfolio.

In [ ]:
%pip install sbti-finance-tool

In [ ]:
%load_ext autoreload
%autoreload 2
import SBTi
from SBTi.data.excel import ExcelProvider
from SBTi.portfolio_aggregation import PortfolioAggregationMethod
from SBTi.portfolio_coverage_tvp import PortfolioCoverageTVP
from SBTi.temperature_score import TemperatureScore
from SBTi.interfaces import ETimeFrames, EScope
from datetime import datetime
import pandas as pd

### Download resources for Google Colab environment


In [ ]:
# Download the dummy data
import urllib.request
import os

if not os.path.isdir("data"):
    os.mkdir("data")
if not os.path.isfile("data/data_provider_example.xlsx"):
    urllib.request.urlretrieve("https://github.com/ScienceBasedTargets/SBTi-finance-tool/raw/main/examples/data/data_provider_example.xlsx", "data/data_provider_example.xlsx")
if not os.path.isfile("data/example_portfolio.csv"):
    urllib.request.urlretrieve("https://github.com/ScienceBasedTargets/SBTi-finance-tool/raw/main/examples/data/example_portfolio.csv", "data/example_portfolio.csv")
if not os.path.isfile("utils.py"):
    urllib.request.urlretrieve("https://github.com/ScienceBasedTargets/SBTi-finance-tool/raw/main/examples/utils.py", "utils.py")

try:  # Import statement when run in Google Colabs
    from utils import  print_aggregations, print_grouped_scores, anonymize, print_percentage_default_scores
except:  # Import statement when run locally
    from examples.utils import print_aggregations, print_grouped_scores, anonymize, print_percentage_default_scores

#### Select reporting date for temperature score calculation
Enter the **reporting date** for time-frame classification. Per the CDP/WWF V1 Temperature Rating Methodology
(Section 2.1.4), time frames are "forward looking" relative to the reporting year:
short-term (≤4 years), mid-term (5–15 years), long-term (15–30 years).

This date also determines whether a target is considered "current" (end year >= reporting year).

In [ ]:
year = 2025  # enter the reporting year
month = 12   # enter the month
day = 31     # enter the day

reporting_date = datetime(year, month, day)
print(f"Reporting date: {reporting_date.strftime('%Y-%m-%d')}")

**Create a data provider**

By running the following cell, you load our example data. If you want to score your own portfolio, this is where you can upload data
from your data provider or your own data lake. Please refer to [Data Requirements](https://sciencebasedtargets.github.io/SBTi-finance-tool/DataRequirements.html#) for a
full description of the data and formats needed.

By specifying a data provider, you can connect to the data source of your choice, e.g. CDP, Urgentem, your own data lake, etc. In this
example, the provider data is located in an Excel file.
Per default we use the example data downloaded in the previous step.
To replace the example data with your own data, first place your file in the data folder:
- When using Google Colab: click the folder logo in the left pane; hover your mouse of the 'data' folder; click the three dots; click upload; select your file.
- When running this notebook locally: place your file in the 'data' folder that is located in the same directory as this notebook.

Next, change the file location between the quotes below: replace 'data_provider_example' with your_filename (don't remove the double quotes in the cell below).

In [ ]:
provider = ExcelProvider(path="data/data_provider_example.xlsx")

**Load your portfolio**

In this example, the portfolio data is stored as a CSV file. If you wish to replace the portfolio file with your own portfolio file,
your CSV file should at least have a "company_id" column (the identifier of the company) and a "investment value" column (the amount
of money invested in each of the companies in your portfolio e.g. the value of the shares you hold). Please refer to [Data Requirements](https://sciencebasedtargets.github.io/SBTi-finance-tool/DataRequirements.html#) for a full description of the data and formats needed.
To change the portfolio file, follow the same steps as for the data provider, described above.
Next, change the file location between the quotes below: replace '`example_portfolio`' with your_filename (don't remove the double quotes in the cell below).

In [ ]:
portfolio = pd.read_csv("data/example_portfolio.csv", encoding="iso-8859-1")
# Rename columns if needed to match expected format
if 'isin' in portfolio.columns and 'company_isin' not in portfolio.columns:
    portfolio = portfolio.rename(columns={'isin': 'company_isin'})
if 'lei' in portfolio.columns and 'company_lei' not in portfolio.columns:
    portfolio = portfolio.rename(columns={'lei': 'company_lei'})

The next cell displays the first 5 rows of data in your file.

In [ ]:
portfolio.head(5)

To load the data from the data provider, we need to convert the data of the portfolio.

In [ ]:
companies = SBTi.utils.dataframe_to_portfolio(portfolio)


## Select calculation options
Configure computational settings for the temperature score. 
You can change the default score (fallback score in the code) and aggregation method below. The values specified here are the recommended defaults by the SBTi. If you want to overwrite the recommended settings see the comments for all the options per setting

In [ ]:
# Settings                                              # Other Options: 
time_frames = [SBTi.interfaces.ETimeFrames.MID]         # reporting to TVT requires 'MID'
scopes = [EScope.S1S2, EScope.S1S2S3]                   # reporting to TVT requires 'S1S2' and 'S1S2S3'
fallback_score = 3.2                                    # The options of the default score are 3.2, 3.9 or 4.5
aggregation_method = PortfolioAggregationMethod.WATS    # Options for the aggregation method are WATS, TETS, AOTS, MOTS, EOTS, ECOTS, and ROTS.
grouping = ['sector']                                   # reporting to TVT requires 'sector'

In [ ]:
temperature_score = TemperatureScore(
    time_frames=time_frames,
    scopes=scopes,
    fallback_score=fallback_score,
    aggregation_method=aggregation_method,
    grouping=grouping
)

## Calculate company and portfolio temperature scores

In [ ]:
amended_portfolio = temperature_score.calculate(data_providers=[provider], portfolio=companies, reporting_date=reporting_date)
aggregated_scores = temperature_score.aggregate_scores(amended_portfolio)
amended_portfolio.head(5) # display top 5 rows

## Checks
In the cell below all figures are printed in a text format one can copy and send for validation purposes to the TVT. 
- Calculation settings
- Portfolio coverage
- Portfolio temperature score
- Percentage covered by targets vs. default scores
- Temperature Score for each sector



In [ ]:

# Portfolio temperature score
print('Calculation settings:')
print()
print('Aggregation method:\t {}'.format(str(aggregation_method).split('.')[1]))
print('Default score: \t\t {:.2f}'.format(fallback_score))
print()

# Portfolio coverage: Percentage of SBTs in portfolio
portfolio_coverage_tvp = PortfolioCoverageTVP()
coverage = portfolio_coverage_tvp.get_portfolio_coverage(amended_portfolio.copy(), PortfolioAggregationMethod.WATS)
print("Portfolio coverage is {c:.2f}%".format(c=coverage))
print()

print('Portfolio Temperature scores:\n')
print_aggregations(aggregated_scores)
print()

# Percentage of score based on default score
print('Percentage of score based on default:\n')
print_percentage_default_scores(aggregated_scores)
print()

# Temperature score per sector
print('Temperature scores per sector:')
print_grouped_scores(aggregated_scores)




## Save data for SBTi target validation
In order for the targets to be validated by SBTi, you can save your data locally. Before saving, choose how you want to handle company-identifying information (company name, ISIN, LEI).

### USER INPUT: Choose an anonymization method

Set **ANONYMIZE_MODE** in the next cell to one of three options:

- **"pseudonymize"** (recommended for TVT submissions) -- Replaces real company identifiers with sequential placeholder labels. For example, company names become "Company1", "Company2", etc.; company IDs and ISINs become "C1", "C2", etc.; and LEIs become "L1", "L2", etc. The data relationships between portfolio and provider files are preserved, so the SBTi Target Validation Team can still reproduce your temperature scores without knowing the real company identities.

- **"remove"** -- Drops the company name, ISIN, and LEI columns entirely from the output. Only the company_id column is kept. Use this if you do not need the recipient to reproduce scores at the individual company level.

- **"none"** -- Keeps all company-identifying information as-is. Use this if you are saving the file for your own internal records and do not need to share it externally.

In [ ]:
# ---- USER INPUT ----
# Choose one of: "pseudonymize", "remove", or "none"
ANONYMIZE_MODE = "none"
# --------------------

import copy as copy

if ANONYMIZE_MODE == "pseudonymize":
    # Replace identifiers with sequential labels (Company1/C1/L1, Company2/C2/L2, etc.)
    # This preserves data relationships so scores can be reproduced by the recipient.
    portfolio_out = portfolio.copy()
    provider_out = copy.deepcopy(provider)
    portfolio_out, provider_out = anonymize(portfolio_out, provider_out)
    print("Pseudonymized: company identifiers replaced with placeholder labels (Company1, C1, L1, etc.)")

elif ANONYMIZE_MODE == "remove":
    # Strip identifying columns entirely, keeping only company_id
    portfolio_out = portfolio.copy()
    provider_out = copy.deepcopy(provider)
    pii_columns = ['company_name', 'company_isin', 'company_lei']
    for col in pii_columns:
        if col in portfolio_out.columns:
            portfolio_out.drop(columns=[col], inplace=True)
    for sheet_key in ['fundamental_data', 'target_data']:
        if sheet_key in provider_out.data:
            for col in pii_columns:
                if col in provider_out.data[sheet_key].columns:
                    provider_out.data[sheet_key].drop(columns=[col], inplace=True)
    print("Removed: company name, ISIN, and LEI columns dropped from output.")

elif ANONYMIZE_MODE == "none":
    portfolio_out = portfolio.copy()
    provider_out = copy.deepcopy(provider)
    print("No anonymization applied. Output will contain real company identifiers.")

else:
    raise ValueError("ANONYMIZE_MODE must be 'pseudonymize', 'remove', or 'none'.")

### Save the output files
To store the portfolio and provider data locally, run the cell below. The filenames will reflect which anonymization mode you chose above.

**Google Colab users:** After running the cell below, click the folder icon in the left pane, hover over the output file, click the three dots, and select "Download" to save it to your computer.

**Local users:** The output files will appear in the same folder as this notebook.

In [ ]:
# Set filenames based on anonymization mode
if ANONYMIZE_MODE == "pseudonymize":
    portfolio_filename = 'portfolio_pseudonymized.xlsx'
    provider_filename = 'provider_pseudonymized.xlsx'
elif ANONYMIZE_MODE == "remove":
    portfolio_filename = 'portfolio_anonymized.xlsx'
    provider_filename = 'provider_anonymized.xlsx'
else:
    portfolio_filename = 'portfolio.xlsx'
    provider_filename = 'provider.xlsx'

portfolio_out.to_excel(portfolio_filename, index=False)

with pd.ExcelWriter(provider_filename, engine='openpyxl') as writer:
    provider_out.data['fundamental_data'].to_excel(writer, sheet_name='fundamental_data')
    provider_out.data['target_data'].to_excel(writer, sheet_name='target_data')

print(f"Saved: {portfolio_filename} and {provider_filename}")

If you run the SBTi tool locally, you find the output files in the root folder of this notebook

If you run the SBTi tool from Google Colab, you:
- Click on the files icon in the left pane
- Click the three dots that appear after hovering over the file
- Download the file to your local machine